In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
pwd

'/gdrive'

In [3]:
cd 'My Drive/data'

/gdrive/My Drive/data


In [4]:
import numpy as np
import gensim
import string

import textwrap

from keras.layers import LSTM, CuDNNLSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation, Dropout
from keras.models import Sequential
from keras.callbacks import LambdaCallback, ModelCheckpoint

from keras.utils.data_utils import get_file
from keras.models import load_model
from pickle import load, dump

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec, KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec


Using TensorFlow backend.


In [26]:
print('\nFetching the text...')
path = 'dragonlance_chronicles.txt'


Fetching the text...


In [0]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [28]:
# load document
path = 'dragonlance_chronicles.txt'
docs = load_doc(path)
print(docs[:200])

AN UNLIKELY GROUP OF HEROES …

Tanis Half-Elven, leader of the companions. A skilled fighter who detests fighting, he is tormented by love for two women—the tempestuous swordswoman, Kitiara, and the e


In [0]:
# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [0]:
import string
 
# turn a doc into clean tokens
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

In [46]:
print('\nPreparing the sentences...')
max_sentence_len = 50
#with open(path) as file_:
#    docs = file_.readlines()
#sentences = [[word for word in doc.lower().translate(str.maketrans('', '', string.punctuation)).split()[:max_sentence_len]] \
#             for doc in docs if doc.strip('\n')]
raw_sentences = split_into_sentences(docs)
sentences = []
for each_sentence in raw_sentences:
    cleaned_sentence = clean_doc(each_sentence)
    if cleaned_sentence:
        sentences.append(cleaned_sentence[:max_sentence_len])


Preparing the sentences...


In [0]:
#max_sentence_len = max([len(one_sentence) for one_sentence in sentences])
#print(max_sentence_len)

In [48]:
#print('Num sentences:', len(sentences))
print(sentences[:2])
print('Total sentences: %d' % len(sentences))
#print('Unique words: %d' % len(set(raw_sentences)))

[['an', 'unlikely', 'group', 'of', 'heroes', 'tanis', 'halfelven', 'leader', 'of', 'the', 'companions'], ['a', 'skilled', 'fighter', 'who', 'detests', 'fighting', 'he', 'is', 'tormented', 'by', 'love', 'for', 'two', 'tempestuous', 'swordswoman', 'kitiara', 'and', 'the', 'enchanting', 'elfmaiden', 'laurana']]
Total sentences: 39423


In [8]:
import gensim.downloader as api
word_vectors = api.load("glove-wiki-gigaword-300")

[==================================================] 100.0% 376.1/376.1MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [49]:
print('\nTraining word2vec...')
# workers=1 will ensure a fully deterministrically-reproducible run, per Gensim docs
word_model = gensim.models.Word2Vec(sentences, size=300, min_count=1, window=5, iter=100, workers=1)
pretrained_weights = word_model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)


Training word2vec...
Result embedding shape: (15462, 300)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  after removing the cwd from sys.path.


In [50]:
print('Checking similar words:')
for word in ['dragon', 'kender', 'magic', 'sword', 'war']:
    most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in word_model.most_similar(word)[:8])
    print('  %s -> %s' % (word, most_similar))

Checking similar words:
  dragon -> supervised (0.43), dragons (0.41), reconnoitering (0.41), watchso (0.37), dragan (0.36), cyan (0.35), relay (0.35), marblesized (0.34)
  kender -> tas (0.37), tasslehoff (0.37), halfelf (0.33), dwarf (0.31), gnosh (0.31), gnome (0.29), plainsman (0.29), battleaxe (0.29)
  magic -> spells (0.42), history (0.38), spell (0.37), components (0.37), concentration (0.36), protection (0.35), strength (0.34), carcasses (0.34)
  sword -> blade (0.48), dagger (0.44), longsword (0.40), mace (0.39), axe (0.39), dragonhelm (0.39), sward (0.39), knife (0.38)
  war -> order (0.36), leadership (0.33), world (0.33), ergoth (0.33), land (0.32), journey (0.32), armies (0.32), knighthood (0.30)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
def word2idx(word):
    return word_model.wv.vocab[word].index
def idx2word(idx):
    return word_model.wv.index2word[idx]

In [52]:
print('\nPreparing the data for LSTM...')
train_x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
train_y = np.zeros([len(sentences)], dtype=np.int32)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence[:-1]):
        train_x[i, t] = word2idx(word)
    train_y[i] = word2idx(sentence[-1])
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)


Preparing the data for LSTM...
train_x shape: (39423, 50)
train_y shape: (39423,)


In [53]:
print('\nTraining LSTM...')
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))

model.add(CuDNNLSTM(units=emdedding_size, return_sequences=True))
model.add(Dropout(0.2))

model.add(CuDNNLSTM(units=emdedding_size))
model.add(Dropout(0.2))

model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


Training LSTM...





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [54]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         4638600   
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, None, 300)         722400    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 300)         0         
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 300)               722400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 15462)             4654062   
_________________________________________________________________
activation_1 (Activation)    (None, 15462)            

In [0]:
def sample_word(preds, temperature=1.0):
    if temperature <= 0:
        return np.argmax(preds)
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def generate_next(text, num_generated=10, temperature=1.0):
    word_idxs = [word2idx(word) for word in text.lower().split()]
    for i in range(num_generated):
        prediction = model.predict(x=np.array(word_idxs))
        idx = sample_word(prediction[-1], temperature)
        word_idxs.append(idx)
    return ' '.join(idx2word(idx) for idx in word_idxs)

In [0]:
def on_epoch_end(epoch, _):
    print('\nGenerating text after epoch: %d' % epoch)
    texts = [
            'deep convolutional',
            'simple and effective',
            'a nonconvex',
            'a',
            ]
    for text in texts:
        generated_text = generate_next(text)
        print('%s... -> %s' % (text, generated_text))

In [0]:
# define the checkpoint
filepath="textgen_dragonlance_word2vec_wgt_improvement-{epoch:02d}-{loss:.4f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [59]:
model.fit(train_x, train_y,
          batch_size=128,
          epochs=100,
#          callbacks=callbacks_list)
#          callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
39423/39423 [==============================] - 15s 374us/step - loss: 7.8810

Epoch 00001: loss improved from inf to 7.88100, saving model to textgen_dragonlance_word2vec_wgt_improvement-01-7.8810.h5
Epoch 2/100
39423/39423 [==============================] - 10s 266us/step - loss: 7.5018

Epoch 00002: loss improved from 7.88100 to 7.50180, saving model to textgen_dragonlance_word2vec_wgt_improvement-02-7.5018.h5
Epoch 3/100
39423/39423 [==============================] - 11s 269us/step - loss: 7.4659

Epoch 00003: loss improved from 7.50180 to 7.46594, saving model to textgen_dragonlance_word2vec_wgt_improvement-03-7.4659.h5
Epoch 4/100
39423/39423 [==============================] - 11s 272us/step - loss: 7.4528

Epoch 00004: loss improved from 7.46594 to 7.45275, saving model to textgen_dragonlance_word2vec_wgt_improvement-04-7.4528.h5
Epoch 5/100
39423/39423 [=====================

In [0]:
# save the model to file
model.save('textgen_dragonlance_word2vec_cleaner_docs_model.h5')

In [61]:
# save trained word vector
word_model.wv.save_word2vec_format('textgen_dragonlance_word2vec_cleaner_docs_vectors.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# load the model
model = load_model('textgen_dragonlance_word2vec_cleaner_docs_model.h5')

In [0]:
# load trained word vector
word_model = KeyedVectors.load_word2vec_format('textgen_dragonlance_word2vec_cleaner_docs_vectors.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:

#glove_file = datapath('/home/ubuntu/downloads/glove/glove.6B.50d.txt')
glove_file = datapath('/home/ringoshin/Downloads/data/glove.6B.100d.txt')
tmp_file = get_tmpfile("glove_word2vec.txt")

# call glove2word2vec script
# default way (through CLI): python -m gensim.scripts.glove2word2vec --input <glove_file> --output <w2v_file>
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_file, tmp_file)

model = KeyedVectors.load_word2vec_format(tmp_file)

In [0]:
# generate a sequence of characters with a language model
def generate_sentence(text, num_generated=40, temperature=1.0):
    generated_text = generate_next(text, num_generated, temperature)

#    print("\n--------------------------------------------------------\n")
    print('\n----- temperature:', temperature)
    print('%s... ->' %(text))
    print('  \"', textwrap.fill('%s' % (generated_text), 80), '\"')
    print("\n--------------------------------------------------------\n")
    
    return generated_text

In [63]:
#start_index = random.randint(0, len(text) - maxlen - 1)
#sentence = text[start_index: start_index + maxlen]

temperature_table = [0.2, 1.0, 1.2] 

texts = [
    'for in ages deep',
    'kender shrugged',
    'raistlin ',
    'a',
  ]
  
for temperature in temperature_table:
    for text in texts:
        _ = generate_sentence(text, max_sentence_len, temperature)


----- temperature: 0.2
for in ages deep... ->
  " for in ages deep bathing corners intervened voyage sullenly enormous remarkable
fainted dead shrug shrug shrug shrug shrug stairway southgate creak bunnies
board reinforcements spangles city edge creak jealous sack negotiable knowledge
shut civilization caves caves caves caves caves particular slightly spangles
city companions spangles city companions spangles city edge cities jubilantly or
canyon "

--------------------------------------------------------


----- temperature: 0.2
kender shrugged... ->
  " kender shrugged tome likely centaurs spangles city edge creak ugly judgement
order very spangles city companions spangles city berem insolence jetties
spangles city edge creak bunnies board look feather winning head win clatter
beneath where rash blinded touched it shadowy open spangles city companions
spangles city edge creak jealous t depart glow "

--------------------------------------------------------


----- temperature: 0.2
r

In [78]:
_ = generate_sentence('we', max_sentence_len, 0.5)


----- temperature: 0.5
we... ->
  " we adventures intrigued afore indulgently snag tossed cheers build animals ships
yourself numbly cast magnitude officers warily elk panicstricken storm low
dargaard drunken teeth piety drained ruled insinuate stairwell rock written
delicious one warily ceased edge dargaard spooks swooped turned swordblades or
turmoil emotions fearful demands doom clatter ventilation gakhan barrels "

--------------------------------------------------------

